In [34]:
import requests
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime 
from bs4 import BeautifulSoup

In [35]:
log_file = "code_log.txt"
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
table_name = 'Largest_banks'
exchange_rate_csv_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
attribute_list = ['Name', "MC_USD_Billion"]
csv_path = './Largest_banks_data.csv'

conn = sqlite3.connect(db_name)

In [36]:
def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthcar_model-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [37]:
page = requests.get(url).text
print(page)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled" lang="en" dir="ltr">
<head><script type="text/javascript" src="//web-static.archive.org/_static/js/bundle-playback.js?v=6XRi73ky" charset="utf-8"></script>
<script type="text/javascript" src="//web-static.archive.org/_static/js/wombat.js?v=txqj7nKC" charset="utf-8"></script>
<script>window.RufflePlayer=window.RufflePlayer||{};window.RufflePlayer.config={"autoplay":"on","unmuteOverlay":"hidden"};</script>
<script type="text/javascript" src="//web-static.archive.org/_static/js/ruffle.js"></script>
<script type="text/javascript">

In [38]:
data = BeautifulSoup(page,'html.parser')
print(data)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-disabled" dir="ltr" lang="en">
<head><script charset="utf-8" src="//web-static.archive.org/_static/js/bundle-playback.js?v=6XRi73ky" type="text/javascript"></script>
<script charset="utf-8" src="//web-static.archive.org/_static/js/wombat.js?v=txqj7nKC" type="text/javascript"></script>
<script>window.RufflePlayer=window.RufflePlayer||{};window.RufflePlayer.config={"autoplay":"on","unmuteOverlay":"hidden"};</script>
<script src="//web-static.archive.org/_static/js/ruffle.js" type="text/javascript"></script>
<script type="text/javascript"

In [8]:
df = pd.DataFrame(columns=attribute_list)
df.head()

,Name,MC_USD_Billion


In [9]:
tables = data.find_all('tbody')
print(tables)

[<tbody><tr>
<th data-sort-type="number">Rank
</th>
<th>Bank name
</th>
<th>Market cap<br/>(US$ billion)
</th></tr>
<tr>
<td>1
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/htt

In [10]:
rows = tables[0].find_all('tr')
print(rows)

[<tr>
<th data-sort-type="number">Rank
</th>
<th>Bank name
</th>
<th>Market cap<br/>(US$ billion)
</th></tr>, <tr>
<td>1
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/https://e

In [32]:
df = pd.DataFrame(columns=["Name", "Market Cap (US$ Billion)"])

for row in rows:
    col = row.find_all('td')
    if (col != []):
        name = col[1].text
        marketcap = col[2].text.strip()
        df = df.append({"Name": name, "Market Cap (US$ Billion)": marketcap}, ignore_index=True)
df

C:\Users\Sandy Lauguico\AppData\Local\Temp\ipykernel_58792\1189201978.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": name, "Market Cap (US$ Billion)": marketcap}, ignore_index=True)
C:\Users\Sandy Lauguico\AppData\Local\Temp\ipykernel_58792\1189201978.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": name, "Market Cap (US$ Billion)": marketcap}, ignore_index=True)
C:\Users\Sandy Lauguico\AppData\Local\Temp\ipykernel_58792\1189201978.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Name": name, "Market Cap (US$ Billion)": marketcap}, ignore_index=True)
C:\Users\Sandy Lauguico\AppData\Local\Temp\ipykernel_58792\1189201978.py:8: FutureWarning: The frame

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase\n,432.92
1,Bank of America\n,231.52
2,Industrial and Commercial Bank of China\n,194.56
3,Agricultural Bank of China\n,160.68
4,HDFC Bank\n,157.91
5,Wells Fargo\n,155.87
6,HSBC Holdings PLC\n,148.90
7,Morgan Stanley\n,140.83
8,China Construction Bank\n,139.82
9,Bank of China\n,136.81


In [33]:
data.prettify()

AttributeError: 'DataFrame' object has no attribute 'prettify'